## Project Penney
Goals:

- For a standard 52-card deck, compute the probability of winning/losing/drawing for player 2
    - Compare probabilities if scoring by totals cards vs tricks
    - Look at all 64-8=56 possible games (minus the 8 ways for players to choose the same sequence)
    - Present the results in a heatmap
    
- Want results to be easily reproducible and verifiable
    - I.e, we want to generate random decks, but will also need to store the seeds
    
- Want to be able to easily debug/test things
    - I.e., if some of our numbers look suspect, be able to run a quick match of e.g, 1000 games.
    
- Want to make it easy to augment the existing data, for example, add an additional million random decks without having to start all the way from the beginning.

- As much as possible, be efficient in terms of runtime and memory usage.

In [33]:
import numpy as np
import src.datagen
import src.helpers
import src.processing
import src.visualizations

from importlib import reload
reload(src.datagen)
reload(src.helpers)

<module 'src.helpers' from '/Users/thomasburkett/Library/CloudStorage/OneDrive-William&Mary/Year 3/Spring 2025/DATA 440 A&W/PROJECTPENNEY/src/helpers.py'>

In [135]:
# Testing the datagen file to ensure that decks are loaded

n = 100000
seed = 42

src.datagen.store_data(n, seed)

In [137]:
# Testing the load_file function from src.processing
filename = 'data/decks_42.npy'

src.processing.load_file(filename)

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 1]])

In [139]:
# Testing the load_file function for file that does not exist
filename = 'data/decks_43.npy'

src.processing.load_file(filename)

File "data/decks_43.npy" does not exist. Try another file.


In [141]:
# Testing the load_file function from src.processing
filename = 'data/decks_42.npy'

deck = src.processing.load_file(filename)
deck

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [1, 1, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 1]])

In [143]:
deck.shape

(101200, 52)

In [145]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Helper function to check if a combination exists in the sequence of cards
def check_combination(sequence, combination):
    """Check if a combination appears in the sequence of cards."""
    return combination in ''.join(['R' if card == 0 else 'B' for card in sequence])

# Simulate a single game of Penney's game
def play_penney_game(player1_comb, player2_comb, deck, num_cards=52):
    """
    Simulate one game of Penney's game.
    player1_comb and player2_comb are the card combinations each player is trying to form.
    The deck is the shuffled deck of cards.
    
    Returns:
        (winner, cards_won): winner is 1 or 2, cards_won is how many cards were won by the winner.
    """
    sequence = []
    for i in range(num_cards):
        card = deck[i]
        sequence.append(card)

        # Check if Player 1's combination appears
        if check_combination(sequence[-len(player1_comb):], player1_comb):
            return 1, i + 1  # Player 1 wins at card i + 1 (1-based index)

        # Check if Player 2's combination appears
        if check_combination(sequence[-len(player2_comb):], player2_comb):
            return 2, i + 1  # Player 2 wins at card i + 1 (1-based index)

    return 0, 0  # No one wins if no combination is found

# Function to run a single simulation and calculate win probabilities
def match_probabilities(decks, player1_comb, player2_comb, num_simulations=10000):
    """
    Run simulations and calculate the win probabilities for player1, player2, and draws.
    
    Args:
        decks: A numpy array of shuffled decks (each deck represents a game).
        player1_comb: The combination player 1 is looking for.
        player2_comb: The combination player 2 is looking for.
        num_simulations: Number of games to simulate to approximate probabilities.
    
    Returns:
        (prob_player1_win, prob_player2_win, prob_draw): probabilities for each outcome.
    """
    results = {"player1_win": 0, "player2_win": 0, "draw": 0}
    
    for i in range(num_simulations):
        deck = np.random.choice([0, 1], size=52)  # Simulating a shuffled deck of 52 cards (Red=0, Black=1)
        
        winner, _ = play_penney_game(player1_comb, player2_comb, deck)
        
        if winner == 1:
            results["player1_win"] += 1
        elif winner == 2:
            results["player2_win"] += 1
        else:
            results["draw"] += 1

    # Calculate the probabilities
    total_games = num_simulations
    prob_player1_win = results["player1_win"] / total_games
    prob_player2_win = results["player2_win"] / total_games
    prob_draw = results["draw"] / total_games

    return prob_player1_win, prob_player2_win, prob_draw

# Example Usage for All 56 Combinations:

# List of all possible combinations (except identical ones)
combinations = [
    ('BBB', 'RBB'), ('BBR', 'RBB'), ('BRB', 'BBR'), ('BRR', 'BBR'), 
    ('RBB', 'RRB'), ('RBR', 'RRB'), ('RRB', 'BRR'), ('RRR', 'BRR')
]

# We will store the results in a dictionary
results_dict = {}

for player1_comb, player2_comb in combinations:
    prob_p1_win, prob_p2_win, prob_draw = match_probabilities(decks=None,  # You can load the actual decks here
                                                                player1_comb=player1_comb,
                                                                player2_comb=player2_comb,
                                                                num_simulations=10000)
    
    # Store the results
    results_dict[(player1_comb, player2_comb)] = {
        "prob_player1_win": prob_p1_win,
        "prob_player2_win": prob_p2_win,
        "prob_draw": prob_draw
    }

# Presenting the Results


In [147]:
player01_comb = 'BBR'
player02_comb = 'RBB'

match_probabilities(deck, player1_comb, player2_comb)

(0.1306, 0.8694, 0.0)

In [149]:
results_dict

{('BBB', 'RBB'): {'prob_player1_win': 0.1244,
  'prob_player2_win': 0.8756,
  'prob_draw': 0.0},
 ('BBR', 'RBB'): {'prob_player1_win': 0.2462,
  'prob_player2_win': 0.7538,
  'prob_draw': 0.0},
 ('BRB', 'BBR'): {'prob_player1_win': 0.3341,
  'prob_player2_win': 0.6659,
  'prob_draw': 0.0},
 ('BRR', 'BBR'): {'prob_player1_win': 0.3277,
  'prob_player2_win': 0.6723,
  'prob_draw': 0.0},
 ('RBB', 'RRB'): {'prob_player1_win': 0.3275,
  'prob_player2_win': 0.6725,
  'prob_draw': 0.0},
 ('RBR', 'RRB'): {'prob_player1_win': 0.336,
  'prob_player2_win': 0.664,
  'prob_draw': 0.0},
 ('RRB', 'BRR'): {'prob_player1_win': 0.2555,
  'prob_player2_win': 0.7445,
  'prob_draw': 0.0},
 ('RRR', 'BRR'): {'prob_player1_win': 0.1288,
  'prob_player2_win': 0.8712,
  'prob_draw': 0.0}}